# ETL (Extract Transform Load)
### Extraer, Transformar, Carga <div>
#### Es un proceso que se utiliza para mover datos de una fuente (Extract), realizar modificaciones en esos datos según sea necesario (Transform), y cargar los datos resultantes en un destino deseado (Load). </div>

***En este archivo se encuentran todas las transformaciones requeridas***

## 1. Preparación de los datos

### 1.1 Importamos librerías  

In [1]:
# Importamos librerias 
import numpy as np 
import pandas as pd 
import seaborn as sns

### 1.2 Carga inicial de los datos

In [2]:
# Cargar el archivo
datos = pd.read_csv("Data/movies_dataset.csv", delimiter= ',', low_memory= False)
data = pd.read_csv("Data/credits.csv")

In [3]:
# Convertirlo a Dataframe
movies = pd.DataFrame(datos)
credits = pd.DataFrame(data)

### 1.3 Preparación de los datos 
***Comenzaremos con el archivo `movies_dataset.csv`***

In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
movies.head(4)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0


Como vista previa de los datos podemos decir:
- Hay columnas anidadas
- Hay valores faltanes
- Verificaremos si hay valores duplicados
- Mayoria de datos en tipo object
- Tenemos 24 carateristicas en este dataset
- Entre otras...
 

In [6]:
# Veamos la cantidad exacta de nulos 
def null(df, decimales=2):
    df_nulos = pd.DataFrame({
        "Numeros de nulos" : df.isnull().sum(),
        "Porcentaje de nulos" : (df.isnull().sum() / df.shape[0]) * 100.0
    })
    df_nulos['Porcentaje de nulos'] = df_nulos['Porcentaje de nulos'].round(decimales).astype(str) + "%"
    return df_nulos

In [7]:
null(movies, decimales=2)

,Numeros de nulos,Porcentaje de nulos
adult,0,0.0%
belongs_to_collection,40972,90.12%
budget,0,0.0%
genres,0,0.0%
homepage,37684,82.88%
id,0,0.0%
imdb_id,17,0.04%
original_language,11,0.02%
original_title,0,0.0%
overview,954,2.1%


In [8]:
# Eliminaremos las columnas no deseadas 
movies = movies.drop(columns = ["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"])

In [9]:
# Eliminaremos las filas con valores nulos en la columna release_date
movies = movies.dropna(subset=["release_date"])

# Rellenamos con 0 los valores faltantes en la columna revenue
movies['revenue'] = movies['revenue'].fillna(0)

In [10]:
# Seguimos !  verificamos que la columna release_date tenga el formato AAAA-mm-dd
movies['release_date'] = pd.to_datetime(movies['release_date'], format = '%Y-%m-%d', errors = 'coerce')
movies['release_date'].sample(4)

32709   2015-08-21
24968   2013-09-06
6771    1977-02-25
42922   2016-09-15
Name: release_date, dtype: datetime64[ns]

In [11]:
# veamos si hay algun valor este como astype datetime y sea Null
print(f"Hay" ,movies['release_date'].isna().sum(), "filas que no esta en astype (tipo de dato) Datetime")

Hay 3 filas que no esta en astype (tipo de dato) Datetime


In [12]:
# Eliminaremos las filas con valores nulos en la columna release_date
movies = movies.dropna(subset=["release_date"])

In [13]:
# Creamos una nueva columna, que contenga solo el año 
movies['release_year'] = movies['release_date'].dt.year.astype('int16')
movies[["release_date", "release_year"]].sample(4)

,release_date,release_year
23182,1988-09-10,1988
22776,2014-02-12,2014
14712,2006-01-01,2006
6935,1984-12-14,1984


In [14]:
# Cambiamos el tipo de dato para las columnas revenue & budget
movies['revenue'] = movies['revenue'].astype('int32')
movies['budget'] = movies['budget'].astype('int32')    

# Creamos una  nueva columna llamada `return` con los campos revenue y budget
movies['return'] = (movies['revenue'] / movies['budget']).where((movies['revenue'] != 0) & (movies['budget'] !=0), 0).astype('float16')
movies[['revenue', 'budget', 'return']].sample(4)

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\dtypes\astype.py:134: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


,revenue,budget,return
15268,0,2300000,0.0
18470,0,0,0.0
25461,0,0,0.0
12633,0,0,0.0


**Ahora solo nos falta desanidar, pero antes realizaremos transformaciones rapidas**

In [15]:
null(movies, decimales=2)

,Numeros de nulos,Porcentaje de nulos
belongs_to_collection,40888,90.11%
budget,0,0.0%
genres,0,0.0%
id,0,0.0%
original_language,11,0.02%
overview,941,2.07%
popularity,0,0.0%
production_companies,0,0.0%
production_countries,0,0.0%
release_date,0,0.0%


In [16]:
# Otras columnas que creo que no son necesarios ni para el deploy 
movies =  movies.drop(columns=['spoken_languages', 'status', 'production_countries', 'belongs_to_collection', 'tagline'])

In [17]:
# LLenaremos con SD significa sin dato a las columnas siguientes
movies['original_language'] = movies['original_language'].fillna('SD')
movies['overview'] = movies['overview'].fillna('SD')
movies['runtime'] = movies['runtime'].fillna('SD')

In [18]:
null(movies, decimales=2)

,Numeros de nulos,Porcentaje de nulos
budget,0,0.0%
genres,0,0.0%
id,0,0.0%
original_language,0,0.0%
overview,0,0.0%
popularity,0,0.0%
production_companies,0,0.0%
release_date,0,0.0%
revenue,0,0.0%
runtime,0,0.0%


In [19]:
# Cambiamos de nombre de id a id_movie
movies.rename(columns={'id' : 'id_movie'}, inplace = True)

In [20]:
# convertimos la cadena de texto en un diccionario
movies['genres'] = movies['genres'].apply(eval)

# Ahora desanidamos las columnas genres, production_companies
genres = movies.explode("genres").reset_index()
genres = pd.concat([genres.drop(columns="genres"), genres['genres'].apply(pd.Series)], axis=1)
genres

,index,budget,id_movie,original_language,overview,popularity,production_companies,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return,id,name,0
0,0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,373554033,81.0,Toy Story,7.7,5415.0,1995,12.453125,16.0,Animation,NaN
1,0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,373554033,81.0,Toy Story,7.7,5415.0,1995,12.453125,35.0,Comedy,NaN
2,0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,373554033,81.0,Toy Story,7.7,5415.0,1995,12.453125,10751.0,Family,NaN
3,1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15,262797249,104.0,Jumanji,6.9,2413.0,1995,4.042969,12.0,Adventure,NaN
4,1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15,262797249,104.0,Jumanji,6.9,2413.0,1995,4.042969,14.0,Fantasy,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93415,45463,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]",2003-08-01,0,90.0,Betrayal,3.8,6.0,2003,0.000000,28.0,Action,NaN
93416,45463,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]",2003-08-01,0,90.0,Betrayal,3.8,6.0,2003,0.000000,18.0,Drama,NaN
93417,45463,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,"[{'name': 'American World Pictures', 'id': 6165}]",2003-08-01,0,90.0,Betrayal,3.8,6.0,2003,0.000000,53.0,Thriller,NaN
93418,45464,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,"[{'name': 'Yermoliev', 'id': 88753}]",1917-10-21,0,87.0,Satan Triumphant,0.0,0.0,1917,0.000000,NaN,NaN,NaN


In [21]:
# Renombramos las columnas 
genres = genres.rename(columns={'id' : 'genre_id', 'name' : 'genre_name'})

# Eliminamos las columnas index, 0
genres.drop(columns=['index', 0], inplace=True)

In [22]:
caomp = genres['production_companies'].to_list()
caomp[:4]

["[{'name': 'Pixar Animation Studios', 'id': 3}]",
 "[{'name': 'Pixar Animation Studios', 'id': 3}]",
 "[{'name': 'Pixar Animation Studios', 'id': 3}]",
 "[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Interscope Communications', 'id': 10201}]"]

In [23]:
genres['production_companies'] = genres['production_companies'].apply(eval)
# pasamos de str a dicionario

In [24]:
# Desanidamos
compa = genres.explode('production_companies').reset_index()
compa = pd.concat([compa.drop(columns="production_companies"), pd.json_normalize(compa['production_companies'])], axis = 1)
compa

,index,budget,id_movie,original_language,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count,release_year,return,genre_id,genre_name,name,id
0,0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Toy Story,7.7,5415.0,1995,12.453125,16.0,Animation,Pixar Animation Studios,3.0
1,1,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Toy Story,7.7,5415.0,1995,12.453125,35.0,Comedy,Pixar Animation Studios,3.0
2,2,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033,81.0,Toy Story,7.7,5415.0,1995,12.453125,10751.0,Family,Pixar Animation Studios,3.0
3,3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Jumanji,6.9,2413.0,1995,4.042969,12.0,Adventure,TriStar Pictures,559.0
4,3,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249,104.0,Jumanji,6.9,2413.0,1995,4.042969,12.0,Adventure,Teitler Film,2550.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178363,93415,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0,90.0,Betrayal,3.8,6.0,2003,0.000000,28.0,Action,American World Pictures,6165.0
178364,93416,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0,90.0,Betrayal,3.8,6.0,2003,0.000000,18.0,Drama,American World Pictures,6165.0
178365,93417,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0,90.0,Betrayal,3.8,6.0,2003,0.000000,53.0,Thriller,American World Pictures,6165.0
178366,93418,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0,87.0,Satan Triumphant,0.0,0.0,1917,0.000000,NaN,NaN,Yermoliev,88753.0


In [25]:
# Hacemos la transformaciones corespondientes
compa = compa.rename(columns={'id' : 'company_id', 'name' : 'company_name'})
compa.drop(columns = ['index'], inplace=True)
compa.columns

Index(['budget', 'id_movie', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'title', 'vote_average',
       'vote_count', 'release_year', 'return', 'genre_id', 'genre_name',
       'company_name', 'company_id'],
      dtype='object')

In [26]:
null(compa, decimales=2)

,Numeros de nulos,Porcentaje de nulos
budget,0,0.0%
id_movie,0,0.0%
original_language,0,0.0%
overview,0,0.0%
popularity,0,0.0%
release_date,0,0.0%
revenue,0,0.0%
runtime,0,0.0%
title,0,0.0%
vote_average,0,0.0%


In [27]:
# Vamos a eliminar los valores faltantes 
compa = compa.dropna(subset=["genre_id"])
compa = compa.dropna(subset=["genre_name"])
compa = compa.dropna(subset=["company_name"])
compa = compa.dropna(subset=["company_id"])

In [28]:
def suma_duplicados(df, decimales=2):
    # Identifica las filas duplicadas
    duplicados = df[df.duplicated()]
    
    # Inicializa diccionarios para almacenar los resultados
    num_duplicados = {}
    porcentaje_duplicados = {}
    
    # Itera sobre cada columna
    for col in df.columns:
        # Cuenta el número de duplicados en la columna
        num_duplicados[col] = duplicados[col].count()
        
        # Calcula el porcentaje de duplicados en la columna
        porcentaje_duplicados[col] = (num_duplicados[col] / df.shape[0]) * 100.0
    
    # Crea un DataFrame para los resultados
    df_duplicados = pd.DataFrame({
        "Numero de Duplicados": pd.Series(num_duplicados),
        "Porcentaje de Duplicados": pd.Series(porcentaje_duplicados)
    })
    
    # Convierte el porcentaje a cadena y añade el símbolo de porcentaje
    df_duplicados["Porcentaje de Duplicados"] = df_duplicados["Porcentaje de Duplicados"].round(decimales).astype(str) + "%"
        
    return df_duplicados

In [29]:
suma_duplicados(compa)

,Numero de Duplicados,Porcentaje de Duplicados
budget,98,0.06%
id_movie,98,0.06%
original_language,98,0.06%
overview,98,0.06%
popularity,98,0.06%
release_date,98,0.06%
revenue,98,0.06%
runtime,98,0.06%
title,98,0.06%
vote_average,98,0.06%


In [30]:
# Eliminamos los duplicados 
compa.drop_duplicates(subset=compa.columns, inplace=True)

***Ultimas transformaciones al dataset movies_datasets***

In [31]:
# Cambiamos de nombre a algunas columnas 
compa.rename(columns={'title' : 'name'}, inplace=True)

In [32]:
compa['runtime'].replace(['SD'], 0, inplace=True)

In [41]:
convert_dict ={
    'id_movie'              : 'int16',
    'name'                  : 'string',
    'original_language'     : 'string',
    'release_date'          : 'datetime64[ns]',
    'release_year'          : 'int16',
    'genre_id'              : 'int16',
    'genre_name'            : 'string',
    'popularity'            : 'float32',
    'runtime'               : 'int8',
    'vote_average'          : 'float16',
    'vote_count'            : 'int32',
    'company_id'            : 'int16',
    'company_name'          : 'string',
    'revenue'               : 'int32',
    'budget'                : 'int32',
    'return'                : 'float16',
    'overview'              : 'string'
    
}
movies = compa.astype(convert_dict)

In [42]:
new_column_order = ['id_movie', 'name', 'original_language', 'release_date', 'release_year', 
                    'genre_id', 'genre_name', 'popularity', 'runtime', 'vote_average', 
                    'vote_count', 'company_id', 'company_name', 'revenue', 'budget', 
                    'return', 'overview']

movies = movies.reindex(columns=new_column_order)

In [43]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 157676 entries, 0 to 178365
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   id_movie           157676 non-null  int16         
 1   name               157676 non-null  string        
 2   original_language  157676 non-null  string        
 3   release_date       157676 non-null  datetime64[ns]
 4   release_year       157676 non-null  int16         
 5   genre_id           157676 non-null  int16         
 6   genre_name         157676 non-null  string        
 7   popularity         157676 non-null  float32       
 8   runtime            157676 non-null  int8          
 9   vote_average       157676 non-null  float16       
 10  vote_count         157676 non-null  int32         
 11  company_id         157676 non-null  int16         
 12  company_name       157676 non-null  string        
 13  revenue            157676 non-null  int32        

In [44]:
movies

,id_movie,name,original_language,release_date,release_year,genre_id,genre_name,popularity,runtime,vote_average,vote_count,company_id,company_name,revenue,budget,return,overview
0,862,Toy Story,en,1995-10-30,1995,16,Animation,21.946943,81,7.699219,5415,3,Pixar Animation Studios,373554033,30000000,12.453125,"Led by Woody, Andy's toys live happily in his ..."
1,862,Toy Story,en,1995-10-30,1995,35,Comedy,21.946943,81,7.699219,5415,3,Pixar Animation Studios,373554033,30000000,12.453125,"Led by Woody, Andy's toys live happily in his ..."
2,862,Toy Story,en,1995-10-30,1995,10751,Family,21.946943,81,7.699219,5415,3,Pixar Animation Studios,373554033,30000000,12.453125,"Led by Woody, Andy's toys live happily in his ..."
3,8844,Jumanji,en,1995-12-15,1995,12,Adventure,17.015539,104,6.898438,2413,559,TriStar Pictures,262797249,65000000,4.042969,When siblings Judy and Peter discover an encha...
4,8844,Jumanji,en,1995-12-15,1995,12,Adventure,17.015539,104,6.898438,2413,2550,Teitler Film,262797249,65000000,4.042969,When siblings Judy and Peter discover an encha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178361,30840,Robin Hood,en,1991-05-13,1991,10749,Romance,5.683753,104,5.699219,26,-26558,CanWest Global Communications,0,0,0.000000,"Yet another version of the classic epic, with ..."
178362,-19963,Century of Birthing,tl,2011-11-17,2011,18,Drama,0.178241,104,9.000000,3,19653,Sine Olivia,0,0,0.000000,An artist struggles to finish his work while a...
178363,2222,Betrayal,en,2003-08-01,2003,28,Action,0.903007,90,3.800781,6,6165,American World Pictures,0,0,0.000000,"When one of her hits goes wrong, a professiona..."
178364,2222,Betrayal,en,2003-08-01,2003,18,Drama,0.903007,90,3.800781,6,6165,American World Pictures,0,0,0.000000,"When one of her hits goes wrong, a professiona..."


**Listo !!!**

In [45]:
# Extraemos el resultado como archivo parquet
# movies.to_parquet("movies.parquet")

### 2. Preparación de los datos 
***Terminamos con el archivo `Credits.csv`***

In [46]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [47]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


Como vista previa podemos concluir:
- Todos son datos anidados, menos id
- En las dos columnas anidadas hay muchas columnas, y filas más

In [48]:
cast = credits['cast'].to_list()
cast[1]

"[{'cast_id': 1, 'character': 'Alan Parrish', 'credit_id': '52fe44bfc3a36847f80a7c73', 'gender': 2, 'id': 2157, 'name': 'Robin Williams', 'order': 0, 'profile_path': '/sojtJyIV3lkUeThD7A2oHNm8183.jpg'}, {'cast_id': 8, 'character': 'Samuel Alan Parrish / Van Pelt', 'credit_id': '52fe44bfc3a36847f80a7c99', 'gender': 2, 'id': 8537, 'name': 'Jonathan Hyde', 'order': 1, 'profile_path': '/7il5D76vx6QVRVlpVvBPEC40MBi.jpg'}, {'cast_id': 2, 'character': 'Judy Sheperd', 'credit_id': '52fe44bfc3a36847f80a7c77', 'gender': 1, 'id': 205, 'name': 'Kirsten Dunst', 'order': 2, 'profile_path': '/wBXvh6PJd0IUVNpvatPC1kzuHtm.jpg'}, {'cast_id': 24, 'character': 'Peter Shepherd', 'credit_id': '52fe44c0c3a36847f80a7ce7', 'gender': 0, 'id': 145151, 'name': 'Bradley Pierce', 'order': 3, 'profile_path': '/j6iW0vVA23GQniAPSYI6mi4hiEW.jpg'}, {'cast_id': 10, 'character': 'Sarah Whittle', 'credit_id': '52fe44bfc3a36847f80a7c9d', 'gender': 1, 'id': 5149, 'name': 'Bonnie Hunt', 'order': 4, 'profile_path': '/7spiVQwmr

In [49]:
# Antes de desanidar en cada columna anidada hay un id, entonces cambiaremos eso
credits = credits.rename(columns={'id' : 'id_movie'})

In [50]:
# Cambiamos de str a diccionario a la columna cast
credits['cast'] = credits['cast'].apply(eval)

In [51]:
# Desanidamos 
cast = credits.explode('cast').reset_index()
cast = pd.concat([cast.drop(columns="cast"), cast['cast'].apply(pd.Series)], axis=1)
cast

,index,crew,id_movie,cast_id,character,credit_id,gender,id,name,order,profile_path,0
0,0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,NaN
1,0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,NaN
2,0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,NaN
3,0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg,NaN
4,0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
564887,45474,"[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,3.0,,52fe4ea59251416c7515d7d9,1.0,1090923.0,Nathalie Lissenko,1.0,None,NaN
564888,45474,"[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,4.0,,52fe4ea59251416c7515d7dd,2.0,1136422.0,Pavel Pavlov,2.0,None,NaN
564889,45474,"[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,5.0,,52fe4ea59251416c7515d7e1,0.0,1261758.0,Aleksandr Chabrov,3.0,None,NaN
564890,45474,"[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506,6.0,,52fe4ea59251416c7515d7e5,1.0,29199.0,Vera Orlova,4.0,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg,NaN


In [52]:
# Realizamos las transformaciones pertinentes
cast.drop(columns=['index', 0, 'crew', 'cast_id', 'character', 'order', 'profile_path'], inplace= True)

cast = cast.rename(columns={'id' : 'actor_id', 'name' : 'name_actor'})

In [53]:
null(cast, decimales=2)

,Numeros de nulos,Porcentaje de nulos
id_movie,0,0.0%
credit_id,2418,0.43%
gender,2418,0.43%
actor_id,2418,0.43%
name_actor,2418,0.43%


In [54]:
# Eliminamos la columna credit_id
cast.drop(columns=['credit_id'], inplace=True)
# Ahora eliminamos los valores faltantes de todas las filas
cast.dropna(subset=cast.columns, inplace=True)

In [55]:
# Veamos soi hay valores duplicados 
suma_duplicados(cast)

,Numero de Duplicados,Porcentaje de Duplicados
id_movie,1637,0.29%
gender,1637,0.29%
actor_id,1637,0.29%
name_actor,1637,0.29%


In [56]:
# Eliminamos los valroes duplicados 
cast.drop_duplicates(subset=cast.columns, inplace=True)

***Vamos a divir los datos en diferentes archivos csv***

In [57]:
# Vamos por la otra columna anidada 
crew = credits['crew'].to_list()
print(crew[1]) 

# Convertimos de str a dicc
credits['crew'] = credits['crew'].apply(eval) 

[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'department': 'Production', 'gender': 2, 'id': 511, 'job': 'Executive Producer', 'name': 'Larry J. Franco', 'profile_path': None}, {'credit_id': '52fe44bfc3a36847f80a7c89', 'department': 'Writing', 'gender': 2, 'id': 876, 'job': 'Screenplay', 'name': 'Jonathan Hensleigh', 'profile_path': '/l1c4UFD3g0HVWj5f0CxXAvMAGiT.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7cdd', 'department': 'Sound', 'gender': 2, 'id': 1729, 'job': 'Original Music Composer', 'name': 'James Horner', 'profile_path': '/oLOtXxXsYk8X4qq0ud4xVypXudi.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7c7d', 'department': 'Directing', 'gender': 2, 'id': 4945, 'job': 'Director', 'name': 'Joe Johnston', 'profile_path': '/fok4jaO62v5IP6hkpaaAcXuw2H.jpg'}, {'credit_id': '52fe44bfc3a36847f80a7cd7', 'department': 'Editing', 'gender': 2, 'id': 4951, 'job': 'Editor', 'name': 'Robert Dalva', 'profile_path': None}, {'credit_id': '573523bec3a368025100062c', 'department': 'Production', 'gender': 0, 'i

In [58]:
# Desanidamos 
crew = credits.explode('crew').reset_index()
crew = pd.concat([crew.drop(columns='crew'), pd.json_normalize(crew['crew'])], axis=1)
crew

,index,cast,id_movie,credit_id,department,gender,id,job,name,profile_path
0,0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,0,"[{'cast_id': 14, 'character': 'Woody (voice)',...",862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...,...,...,...
465080,45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...",67758,52fe4776c3a368484e0c8399,Sound,0.0,549356.0,Original Music Composer,Richard McHugh,None
465081,45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...",67758,52fe4776c3a368484e0c839f,Camera,2.0,58818.0,Director of Photography,João Fernandes,None
465082,45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...",227506,533bccebc3a36844cf0011a7,Directing,0.0,1085341.0,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg
465083,45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...",227506,58ebbc26925141281908aa0a,Production,2.0,1195656.0,Producer,Joseph N. Ermolieff,None


In [59]:
# Hacemos las transformaciones correspondientes
crew.drop(columns = ['index', 'profile_path', 'cast', 'credit_id', 'department'], inplace= True)

crew.rename(columns={'id' : 'team_member_id'}, inplace=True)

In [60]:
null(crew, decimales=2)

,Numeros de nulos,Porcentaje de nulos
id_movie,0,0.0%
gender,771,0.17%
team_member_id,771,0.17%
job,771,0.17%
name,771,0.17%


In [61]:
# Eliminamos los nulos
crew.dropna(subset=crew.columns, inplace=True)

In [62]:
suma_duplicados(crew)

,Numero de Duplicados,Porcentaje de Duplicados
id_movie,478,0.1%
gender,478,0.1%
team_member_id,478,0.1%
job,478,0.1%
name,478,0.1%


In [63]:
crew.drop_duplicates(subset=crew.columns, inplace=True)

In [73]:
cast

,id_movie,gender,actor_id,name_actor
0,862,2,31,Tom Hanks
1,862,2,12898,Tim Allen
2,862,2,7167,Don Rickles
3,862,2,12899,Jim Varney
4,862,2,12900,Wallace Shawn
...,...,...,...,...
564886,227506,2,544742,Iwan Mosschuchin
564887,227506,1,1090923,Nathalie Lissenko
564888,227506,2,1136422,Pavel Pavlov
564889,227506,0,1261758,Aleksandr Chabrov


In [71]:
# Hacemos pequeñas transformaciones
convert_dict1 ={
    'id_movie'  : 'int32',
    'gender'    : 'int8',
    'actor_id'  : 'int32'
}
cast = cast.astype(convert_dict1)

In [72]:
cast.info()

<class 'pandas.core.frame.DataFrame'>
Index: 560837 entries, 0 to 564890
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_movie    560837 non-null  int32 
 1   gender      560837 non-null  int8  
 2   actor_id    560837 non-null  int32 
 3   name_actor  560837 non-null  object
dtypes: int32(2), int8(1), object(1)
memory usage: 13.4+ MB


In [64]:
crew

,id_movie,gender,team_member_id,job,name
0,862,2.0,7879.0,Director,John Lasseter
1,862,2.0,12891.0,Screenplay,Joss Whedon
2,862,2.0,7.0,Screenplay,Andrew Stanton
3,862,2.0,12892.0,Screenplay,Joel Cohen
4,862,0.0,12893.0,Screenplay,Alec Sokolow
...,...,...,...,...,...
465080,67758,0.0,549356.0,Original Music Composer,Richard McHugh
465081,67758,2.0,58818.0,Director of Photography,João Fernandes
465082,227506,0.0,1085341.0,Director,Yakov Protazanov
465083,227506,2.0,1195656.0,Producer,Joseph N. Ermolieff


In [77]:
# Vamos a realizar una mask, solo queremos las personas que tiene el trabajo de director
crew = crew.loc[crew['job'] == 'Director']
crew

,id_movie,gender,team_member_id,job,name
0,862,2.0,7879.0,Director,John Lasseter
109,8844,2.0,4945.0,Director,Joe Johnston
122,15602,2.0,26502.0,Director,Howard Deutch
126,31357,2.0,2178.0,Director,Forest Whitaker
141,11862,2.0,56106.0,Director,Charles Shyer
...,...,...,...,...,...
465062,439050,0.0,1182809.0,Director,Hamid Nematollah
465071,111109,0.0,1051381.0,Director,Lav Diaz
465077,67758,2.0,67753.0,Director,Mark L. Lester
465082,227506,0.0,1085341.0,Director,Yakov Protazanov


In [80]:
# Hacemos pequeñas transformaciones
convert_dict2 ={
    'id_movie'          : 'int32',
    'gender'            : 'int8',
    'team_member_id'    : 'int32'
}
crew = crew.astype(convert_dict2)

In [82]:
new_columns = ['id_movie', 'team_member_id', 'job', 'name', 'gender']

crew = crew.reindex(columns=new_columns)

In [83]:
crew.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48999 entries, 0 to 465084
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_movie        48999 non-null  int32 
 1   team_member_id  48999 non-null  int32 
 2   job             48999 non-null  object
 3   name            48999 non-null  object
 4   gender          48999 non-null  int8  
dtypes: int32(2), int8(1), object(2)
memory usage: 1.5+ MB


In [84]:
# Guardamos los datos en archivos parquet diferentes
cast.to_parquet("actores.parquet")
crew.to_parquet("equipo.parquet")